In [ ]:
# Define Prompt Template
template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
prompt.format(context="I am a researcher", question="How can I write a paper?")

# Define chain
parser = StrOutputParser()
chain1 = prompt | model | parser 
#chain_goal.invoke({
#    "context":"I am a researcher",
#    "question": "How can I write a paper?"
#})

# LATER (Connecting the chain to another chain)
source_spatial_context = "source context"
target_spatial_context = "target context"

### simpler chain
prompt_action = ChatPromptTemplate.from_template(
    "A source room with context of {source_spatial_context} is conducting {goalstep}. Your space has target context of {target_spatial_context}. You want to conduct the same goals as the source room with current spatial settings. What are the actions that must be taken?" 
)

chain_action = (
    {"source_spatial_context": itemgetter(source_spatial_context), "goalstep": chain1, "target_spatial_context": itemgetter(target_spatial_context)} | prompt_action | model | parser
)